In [ ]:
import pandas as pd

In [1]:
organization_path = '../Dataset/bulk_export/organizations.csv'  
organization_data = pd.read_csv(organization_path)

# Step 2: Manipulate or analyze the data as needed
# For example, you can print the first few rows of the DataFrame using .head()
# print("First 5 rows of the DataFrame:")
# print(organization_data.head())

# Or you can perform some basic analysis, such as getting summary statistics using .describe()
# print("\nSummary statistics of the DataFrame:")
# print(organization_data.describe())

organization_data_num_rows = organization_data.shape[0]
print("Number of Organization in the Organization.csv:", organization_data_num_rows)
organization_data_num_columns = organization_data.shape[1] 
print("Number of columns in the Organization.csv:", organization_data_num_columns)
organization_data_column_names = organization_data.columns
print("Column names in the Organization.csv:")
for name in organization_data_column_names:
    print(name)

/Users/aldemarogonzalez/anaconda3/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
/var/folders/2r/b60vp1dj2zn0rg5yg_gyy0zr0000gq/T/ipykernel_62884/3231300853.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


Number of Organization in the Organization.csv: 3347706
Number of columns in the People.csv: 41
Column names in the Organization.csv:
uuid
name
type
permalink
cb_url
rank
created_at
updated_at
legal_name
roles
domain
homepage_url
country_code
state_code
region
city
address
postal_code
status
short_description
category_list
category_groups_list
num_funding_rounds
total_funding_usd
total_funding
total_funding_currency_code
founded_on
last_funding_on
closed_on
employee_count
email
phone
facebook_url
linkedin_url
twitter_url
logo_url
alias1
alias2
alias3
primary_role
num_exits


In [8]:
people_path = '../Dataset/bulk_export/people.csv'  # Replace 'your_file.csv' with the actual path to your CSV file
people_data = pd.read_csv(people_path)
people_data_num_rows = people_data.shape[0] 
print("Number of People in the People.csv:", people_data_num_rows)
people_data_num_columns = people_data.shape[1] 
print("Number of columns in the People.csv:", people_data_num_columns)
people_data_column_names = people_data.columns
print("Column names in the People.csv:")
for name in people_data_column_names:
    print(name)

Number of People in the People.csv: 1854387
Number of columns in the People.csv: 22
Column names in the People.csv:
uuid
name
type
permalink
cb_url
rank
created_at
updated_at
first_name
last_name
gender
country_code
state_code
region
city
featured_job_organization_uuid
featured_job_organization_name
featured_job_title
facebook_url
linkedin_url
twitter_url
logo_url


In [9]:
funding_path = '../Dataset/bulk_export/funding_rounds.csv'
funding_data = pd.read_csv(funding_path)
funding_data_num_rows = funding_data.shape[0] 
print("Number of funding in the funding.csv:", funding_data_num_rows)
funding_data_num_columns = funding_data.shape[1]
print("Number of columns in the funding.csv:", funding_data_num_columns)
funding_data_column_names = funding_data.columns
print("Column names in the funding.csv:")
for name in funding_data_column_names:
    print(name)

Number of funding in the funding.csv: 631092
Number of columns in the DataFrame: 24
Column names in the DataFrame:
uuid
name
type
permalink
cb_url
rank
created_at
updated_at
country_code
state_code
region
city
investment_type
announced_on
raised_amount_usd
raised_amount
raised_amount_currency_code
post_money_valuation_usd
post_money_valuation
post_money_valuation_currency_code
investor_count
org_uuid
org_name
lead_investor_uuids


Filtering the companies

I filtered companies using their creation date and removed the ones acquired, closed or that went public by an IPO during the Warmup window, 

In [3]:
# Convert 'founded_on' to datetime if it's not already
organization_data['founded_on'] = pd.to_datetime(organization_data['founded_on'])

# Define the start and end dates of the warm-up window
warm_up_start_date = pd.Timestamp('2015-12-01')
warm_up_end_date = pd.Timestamp('2018-12-31')

# Filter companies founded during the warm-up window
organization_data_filtered = organization_data[(organization_data['founded_on'] >= warm_up_start_date) & (organization_data['founded_on'] <= warm_up_end_date)]

# Remove companies that were acquired, closed, or went public by an IPO during the warm-up window
organization_data_filtered = organization_data_filtered[~organization_data_filtered['status'].isin(['acquired', 'closed', 'ipo'])]

# At this point, organization_data_filtered contains companies that were founded during the warm-up window and are not in the acquired, closed, or IPO status during that period


In [ ]:
organization_data_filtered.to_csv('../Dataset/filtered_data/filtered_organization.csv', index=False)


Now I filtered the companies which closed a funding round above Series B or above, which is not interesting for Plug and Play investments

Step 2: Merge the DataFrames
Next, merge the two DataFrames on the specified columns.

In [ ]:
organization_funding_merged = pd.merge(funding_data, organization_data_filtered, left_on='org_uuid', right_on='uuid')


Filter Out Series B or Above
After merging, filter out organizations that have raised a Series B or above. Assuming the funding round information is stored in a column named round_type in the funding_rounds DataFrame, you can do this by excluding rows where round_type is in a list of Series B or above.

In [ ]:
# Define the series that are Series B or above
series_b_or_above = ['series_b', 'series_c', 'series_d', 'series_e', 'series_f', 'series_g']

# Filter out organizations that have raised a Series B or above
organization_data_filtered_by_round = organization_funding_merged[~organization_funding_merged['round_type'].isin(series_b_or_above)]

In [ ]:
# Save the filtered DataFrame to a new CSV file
organization_data_filtered_by_round.to_csv('../Dataset/filtered_data/filtered_organizations_by_rounds.csv', index=False)